In [1]:
from datetime import datetime

import pandas as pd
import numpy as np
from scipy.optimize import Bounds, LinearConstraint, differential_evolution

#### Данные о комнатах и подразделениях:

In [2]:
rooms = pd.read_csv('data/rooms.csv', index_col='room')
rooms

,capacity
room,
1,6
2,3
3,7


In [3]:
staff = pd.read_csv('data/staff.csv', index_col='department')
staff

,size
department,
A,4
B,6
C,3
D,2


##### Функции цели/ограничений для датафреймов:

In [4]:
def explode_distr(distr: pd.DataFrame, rooms: pd.DataFrame, staff: pd.DataFrame) -> pd.DataFrame:
    df = rooms.reset_index().merge(staff.reset_index(), how='cross')
    df = df.merge(distr, how='left', on=['department', 'room'])
    df = df.fillna(0)
    df['count'] = df['count'].astype(int)
    return df
    

def score_for_df(distr: pd.DataFrame) -> float:
    return (distr.groupby('department')['room'].nunique()).mean()


def constraint_capacity_for_df(distr: pd.DataFrame, rooms: pd.DataFrame) -> bool:
    actual = distr.groupby('room')['count'].sum()
    merged = rooms.merge(actual, left_index=True, right_index=True)
    return all(merged['capacity'] >= merged['count'])


def constraint_department_size_for_df(distr: pd.DataFrame, staff: pd.DataFrame) -> bool:
    actual = distr.groupby('department')['count'].sum()
    merged = staff.merge(actual, left_index=True, right_index=True)
    return all(merged['size'] == merged['count'])

#### "Исходное" распределение работников:

In [5]:
initial_distribution = pd.read_csv('data/initial_distribution.csv')

print(f"score: {score_for_df(initial_distribution)}, constraint_capacity: {constraint_capacity_for_df(initial_distribution, rooms)}, constraint_department_size: {constraint_department_size_for_df(initial_distribution, staff)}")
initial_distribution

score: 1.5, constraint_capacity: True, constraint_department_size: True


,room,department,count
0,1,A,4
1,1,B,2
2,2,B,3
3,3,C,3
4,3,D,2
5,3,B,1


#### Вариант "идеального" распределения работников:

In [6]:
hypothesis = pd.read_csv('data/hypothesis.csv')

print(f"score: {score_for_df(hypothesis)}, constraint_capacity: {constraint_capacity_for_df(hypothesis, rooms)}, constraint_department_size: {constraint_department_size_for_df(hypothesis, staff)}")
hypothesis

score: 1.0, constraint_capacity: True, constraint_department_size: True


,room,count,department
0,1,4,A
1,1,2,D
2,2,3,C
3,3,6,B


#### Подготовка к оптимизации

In [7]:
initial_distribution_exploded = explode_distr(initial_distribution, rooms, staff)
initial_distribution_exploded

,room,capacity,department,size,count
0,1,6,A,4,4
1,1,6,B,6,2
2,1,6,C,3,0
3,1,6,D,2,0
4,2,3,A,4,0
5,2,3,B,6,3
6,2,3,C,3,0
7,2,3,D,2,0
8,3,7,A,4,0
9,3,7,B,6,1


#### Постановка ограничений

##### Границы переменных (`lb <= x <= ub`):

In [8]:
bounds = Bounds(lb=0, ub=[min(row['capacity'], row['size']) for _, row in initial_distribution_exploded.iterrows()])
bounds

Bounds(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([4, 6, 3, 2, 3, 3, 3, 2, 4, 6, 3, 2]))

##### Ограничение на заполнение комнат (сумма числа работников в комнате 1 не должна превышать вместимость комнаты 1 и т.д.):

In [9]:
constraint_capacity = LinearConstraint(
    A=pd.concat([(initial_distribution_exploded['room'] == room).astype(int) for room in rooms.index], axis=1).values.T,
    lb=0,
    ub=rooms.values.reshape(-1)
)

constraint_capacity.A, constraint_capacity.lb, constraint_capacity.ub, constraint_capacity.A @ initial_distribution_exploded['count'].values

(array([[1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1.]]),
 array([0., 0., 0.]),
 array([6., 3., 7.]),
 array([6., 3., 6.]))

##### Ограничение на равенство сумм числа работников размеру подразделений:

In [10]:
constraint_staff = LinearConstraint(
    A=pd.concat([(initial_distribution_exploded['department'] == dep).astype(int) for dep in staff.index], axis=1).values.T,
    lb=staff.values.reshape(-1),
    ub=staff.values.reshape(-1)
)

constraint_staff.A, constraint_staff.lb, constraint_staff.ub, constraint_staff.A @ initial_distribution_exploded['count'].values

(array([[1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1.]]),
 array([4., 6., 3., 2.]),
 array([4., 6., 3., 2.]),
 array([4., 6., 3., 2.]))

#### Постановка целевой функции и запуск

Целевая функция (минимизируем количество кабинетов на одно подразделение в среднем):

In [11]:
DEPARTMENT_COUNT = len(staff)
DEPARTMENT_INDICES = [list(initial_distribution_exploded[initial_distribution_exploded['department'] == dep].index) for dep in staff.index]

DEPARTMENT_COUNT, DEPARTMENT_INDICES

(4, [[0, 4, 8], [1, 5, 9], [2, 6, 10], [3, 7, 11]])

In [12]:
def objective(x: np.ndarray) -> float:
    return sum((x[idx] != 0).sum() for idx in DEPARTMENT_INDICES) / DEPARTMENT_COUNT


objective(initial_distribution_exploded['count'].values)

1.5

Запуск оптимизатора (https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.differential_evolution.html).<br>
Стратегия `rand1bin` - единственная, которая смогла дать результат `1.0` (ни одно подразделение не разделено на несколько кабинетов).

In [13]:
t0 = datetime.now()
res = differential_evolution(objective,
                             strategy='rand1bin',
                             constraints=[constraint_capacity, constraint_staff], 
                             bounds=bounds, 
                             x0=initial_distribution_exploded['count'].values,
                             maxiter=20_000,
                             integrality=True,
                             seed=1,
                             )
print(f'Time elapsed: {datetime.now() - t0}')

Time elapsed: 0:04:59.935062


##### Результат:

In [14]:
res

           constr: [array([0., 0., 0.]), array([0., 0., 0., 0.])]
 constr_violation: 0.0
              fun: 1.0
            maxcv: 0.0
          message: 'Optimization terminated successfully.'
             nfev: 16966
              nit: 12058
          success: True
                x: array([0., 6., 0., 0., 0., 0., 0., 2., 4., 0., 3., 0.])

In [15]:
optimized_distribution_exploded = initial_distribution_exploded.copy()
optimized_distribution_exploded['count'] = res.x.astype(int)
optimized_distribution = optimized_distribution_exploded[optimized_distribution_exploded['count'] != 0]
optimized_distribution

,room,capacity,department,size,count
1,1,6,B,6,6
7,2,3,D,2,2
8,3,7,A,4,4
10,3,7,C,3,3
